# jobs

> Fill in a module description here

In [ ]:
#| default_exp jobs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import schedule
from triggerkit import core
from typing import Dict, List, Callable, Optional, Any, Union

In [ ]:
#| export
def create(view_name: str, action_names: Union[str, List[str]], job_name: Optional[str] = None):
    """
    Create a job function that fetches data from a view and runs specified actions.
    
    Args:
        view_name: Name of the registered view
        action_names: Name or list of names of registered actions
        job_name: Optional name for the job
        
    Returns:
        Job function
    """
    if isinstance(action_names, str):
        action_names = [action_names]
    
    if not job_name:
        job_name = f"{view_name}_{'_'.join(action_names)}_job"
    
    def job():
        core.logger.info(f"Running job '{job_name}'")
        try:
            # Fetch data from view
            data = execute_query(view_name)
            
            # Run each action
            results = {}
            for action_name in action_names:
                results[action_name] = run_action(action_name, data)
            
            core.logger.info(f"Job '{job_name}' completed successfully")
            return results
        except Exception as e:
            core.logger.error(f"Job '{job_name}' failed: {str(e)}")
            raise
    
    job.__name__ = job_name
    return job

In [ ]:
#| export

def schedule_jobs(config: Dict[str, Any]):
    """
    Schedule jobs based on TOML configuration.
    
    Args:
        config: Dictionary containing configuration sections
        
    Returns:
        None
    """
    jobs = config.get('jobs', [])
    
    for job_config in jobs:
        name = job_config.get('name')
        view_name = job_config.get('view')
        actions = job_config.get('actions', [])
        schedule_str = job_config.get('schedule')
        enabled = job_config.get('enabled', True)
        run_at = job_config.get('run_at')
        
        if not enabled:
            core.logger.info(f"Job '{name}' is disabled, skipping")
            continue
            
        if not all([view_name, actions, schedule_str]):
            core.logger.warning(f"Skipping invalid job configuration: {job_config}")
            continue
        
        job = create_job(view_name, actions, name)
        
        # Parse schedule string and set up schedule
        schedule_parts = schedule_str.split()
        if len(schedule_parts) >= 3 and schedule_parts[0] == "every":
            try:
                interval = int(schedule_parts[1])
                unit = schedule_parts[2].lower()
                
                scheduler = None
                if unit in ('minute', 'minutes'):
                    scheduler = schedule.every(interval).minutes
                elif unit in ('hour', 'hours'):
                    scheduler = schedule.every(interval).hours
                elif unit in ('day', 'days'):
                    scheduler = schedule.every(interval).days
                else:
                    core.logger.warning(f"Unsupported schedule unit: {unit}")
                    continue
                
                # Add specific time if provided
                if run_at and unit in ('day', 'days'):
                    try:
                        hour, minute = map(int, run_at.split(':'))
                        run_time = datetime_time(hour=hour, minute=minute)
                        scheduler = scheduler.at(run_at)
                    except (ValueError, AttributeError):
                        core.logger.warning(f"Invalid run_at time format: {run_at}, expected HH:MM. Using default.")
                
                scheduler.do(job)
                core.logger.info(f"Scheduled job '{name}' for view '{view_name}' with actions {actions} to run {schedule_str}" + 
                           (f" at {run_at}" if run_at and unit in ('day', 'days') else ""))
            except ValueError:
                core.logger.warning(f"Invalid schedule format: {schedule_str}")
                continue
        else:
            core.logger.warning(f"Unsupported schedule format: {schedule_str}")
            continue

In [ ]:
#| export

def run_scheduler():
    """Run the scheduler loop."""
    logger.info("Starting scheduler")
    while True:
        schedule.run_pending()
        time.sleep(1)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()